<a href="https://colab.research.google.com/github/AlbertoEscrivaCastro/03MIAR_04_B_2023-24_Algoritmos-de-Optimizacion/blob/main/Alberto_Escriva_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Alberto Manuel Escrivá Castro <br>
Colab: https://colab.research.google.com/drive/1-AuieV1xBy-r_X17EJAmjqzFq2kFIi0U?usp=sharing <br>
Github: https://github.com/AlbertoEscrivaCastro/03MIAR_04_B_2023-24_Algoritmos-de-Optimizacion/blob/main/Alberto_Escriva_Seminario_Algoritmos.ipynb <br>
Problema:
>1. ~~Sesiones de doblaje~~ <br>
>2. ~~Organizar los horarios de partidos de La Liga~~ <br>
>3. Combinar cifras y operaciones

Descripción del problema:

El problema consiste en **analizar** el siguiente problema y **diseñar** un algoritmo que lo **resuelva**. <br>
- Disponemos de las 9 cifras del 1 al 9 (excluimos el cero) y de los 4 signos básicos de las operaciones fundamentales: suma (+), resta (-), multiplicación (\*) y división (/). <br>
- Debemos **combinarlos alternativamente sin repetir ninguno de ellos** para obtener una cantidad dada. Un ejemplo sería para obtener el 4:
$4+2-6/3*1 = 4$

Debe analizarse el problema para encontrar todos los valores enteros posibles planteando las
siguientes cuestiones:
- ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?
- ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo ?
• Nota: Es posible usar la función de python “eval” para evaluar una expresión.

(*) La respuesta es obligatoria





                                        

In [2]:
import time
import random
import math

## (*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>
## ¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

-----
*Sin tener en cuenta absolutamente ninguna restricción, ni siquiera la de no poder repetir elementos, la siguiente cantidad de posibilidades vendría definida por:*

$9^5*4^4$

In [3]:
# Se establece la cantidad de números que se van a combinar.
n_max             = 9
n_num             = 5
lista_numeros     = list(range( 1 , n_max+1 ))
n_lng             = len( lista_numeros )
lista_operaciones = ['*','+','-','/']

print( lista_numeros )
print( lista_operaciones )

[1, 2, 3, 4, 5, 6, 7, 8, 9]
['*', '+', '-', '/']


In [4]:
posibilidades_sin_restricciones = n_lng**n_num * (n_num-1)**(n_num-1)
print(f'Posibilidades sin considerar NINGUNA restricción: {posibilidades_sin_restricciones}')

Posibilidades sin considerar NINGUNA restricción: 15116544


*Sin embargo, teniendo en cuenta que no se pueden repetir elementos se tendría lo siguiente:*

$\frac{9!}{(9-5)!}*4!$

In [5]:
posibilidades_sin_repeticion = int( (math.factorial( n_lng ) / math.factorial( n_lng-n_num )) * math.factorial( n_num-1 ) )
print(f'Posibilidades sin repeticiones: {posibilidades_sin_repeticion}')

Posibilidades sin repeticiones: 362880


*Se podría intentar incluir la restriccion de compatibilidad de las operaciones para que salga un entero y así reducir aún más la cantidad de posibilidades. Sin embargo, debido a la posibilidad de combinar la división con la multiplicación resulta complicado acotar aún más la cantidad de posibilidades posibles.*

## Modelo para el espacio de soluciones<br>
## (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo. (Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

-------
*En principio, lo mas adecuado tener dos listas independientes con los que trabajar, en los que se tendrán almacenadas las opciones a elegir para cada tipo de elemento. Esto permitirá ir extrayendo elementos de sin repetirlos.*

*Sin embargo, la solución debería almacenarse en una lista o un string, puesto que para ciertas operaciones el órden resulta importante. En particular, un string será más idóneo para aplicarle la función de evaluación de expresiones, pero para manipular internamente los elementos será mejor emplear la lista para generalizar mejor el problema (al generalizar, se podrían tener números de más cífras o símbolos compuestos como '\**').*

*Como ejemplo para la solución, se incluye un a función de generación aleatoria de soluciones.*

In [6]:
def cadena_aleatoria( numeros , simbolos , N ):
  solucion = []
  num_lst  = list( numeros )
  sim_lst  = list( simbolos )

  for i in range( 2*N-1 ):
    if i%2 == 0:
      elemento = random.choice( num_lst )
      num_lst.remove( elemento )
    else:
      elemento = random.choice( sim_lst )
      sim_lst.remove( elemento )

    solucion += str( elemento )

  return solucion


def lista_a_cadena( lista ):
  return ''.join( str (e) for e in lista )

ejemplo = cadena_aleatoria( lista_numeros , lista_operaciones , n_num )

print( ejemplo )
print( lista_a_cadena( ejemplo ) )



['9', '-', '5', '+', '4', '*', '7', '/', '2']
9-5+4*7/2


*Como se verá más adelante, para el algoritmo de fuerza bruta se ha optado en cambio por mantener las listas origen fijas y construir la solución codificada en dos listas independientes en las que en cada una de sus posiciones se guarda la posición que tiene el elemento seleccionado en la lista origen.*

*Por ejemplo, la solución $7/6+1-2*9$,  según la definición de las listas en los apartados anteriores, estaría de ese modo codificada de la siguiente forma:*

$sol\_num = [6,5,0,1,8]$

$sol\_sim = [3,1,2,0]$

*Esto facilita realizar el barrido para el algoritmo de fuerza bruta.*

## Según el modelo para el espacio de soluciones<br>
## (*)¿Cual es la función objetivo?
## (*)¿Es un problema de maximización o minimización?

Respuesta

------
*Se puede definir la función objetivo como el valor absoluto de la diferencia entre el valor buscado y el valor devuelto.*

$f = |y-\hat{y}|$

*Sin embargo, dadas las características del problema, al no poderse realmente eveluar la función objetivo de una forma contínua, este problema, más que un problema de optimización, se trataría de un problema de búsqueda.*

In [7]:
def f_objetivo( objetivo , solucion ):
  return abs( objetivo - eval( solucion ) )

*Dada esta función objetivo, se trataría de un problema de minimización pues se ha definido la función objetivo como un error. Sin embargo, teniendo en cuenta lo comentado previamente, al tratarse más de un problema de búsqueda no sería realmente ni maximización, ni minimización.*

## Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

------
*Para este algoritmo, lo más cómodo será convertir las soluciones en dos listas independientes (una para los números y otra para los símbolos) que almacenarán las posiciones de los elementos seleccionados.*

*Se codifica con interrupción manual pues no se espera que encuentre la solución en un tiempo razonable.*

In [8]:
def decodificar_solucion( num_sol , sim_sol , num_lst , sim_lst ):
  # Se encarga de convertir la solución codificada en una solución interpretable pero aún en formato de lista.
  solucion_lst = [ num_lst[ num_sol[0] ] ]

  for i in range( len( sim_sol ) ) :
    solucion_lst.append( sim_lst[ sim_sol[ i ] ] )
    solucion_lst.append( num_lst[ num_sol[ i+1 ] ] )

  return solucion_lst


def siguiente( len_lista , sol_anterior ):
  # Esta función calcula la siguiente solción en el espacio de soluciones para una de las listas.
  # No tiene en cuenta inguna de las restricciones.
  # Está generalizada para que sirva para cualquier lista y tamaño.
  N             = len( sol_anterior )
  sol_siguiente = [0] * N
  aux           = sol_anterior[-1] + 1

  if aux < len_lista :
    sol_siguiente[-1] = aux
    if N > 1:
      sol_siguiente[:-1] = sol_anterior[:-1].copy()

  else :
    sol_siguiente[-1] = 0
    if N > 1 :
      sol_siguiente[:-1] = siguiente( len_lista , sol_anterior[:-1] )

  return sol_siguiente


def sin_repetidos( lista ):
  validacion = ( len( lista ) == len( set( lista ) ) )

  return validacion


def solucion_entera( solucion ):
  resultado = eval( solucion )

  if isinstance( resultado , int ) :
    validacion = True
  else :
    validacion = resultado.is_integer()

  return validacion


def validaciones( sol_num , sol_sim , sol_completa ):
  sin_repeticiones_num = sin_repetidos( sol_num )
  sin_repeticiones_sim = sin_repetidos( sol_sim )
  sol_entera           = solucion_entera( sol_completa )
  validacion           = ( sin_repeticiones_num and sin_repeticiones_sim and sol_entera )

  return validacion



def fuerza_bruta( numeros , simbolos , N , objetivo , interrupcion = False ):
  tiempo_inicio = time.time()
  tiempos       = [ tiempo_inicio , tiempo_inicio ]
  # Se construye así la inicialización para que al empezar los bucles obteniendo el siguiente se empiece
  # realemente por todo ceros.
  sol_num       = [0]*(N-1) + [-1]
  len_num       = len(numeros)
  len_sim       = len(simbolos)
  detener       = False
  sol_valida    = False
  es_resultado  = False

  while not(   es_resultado     \
           and sol_valida       \
           )                    \
    and sol_num < [len_num-1]*N \
    and not( detener )          :
    # Este bucle barre las opciones de números.
    sol_num = siguiente( len_num , sol_num )
    sol_sim = [0]*(N-2) + [-1]

    while not(   es_resultado         \
             and sol_valida           \
             )                        \
      and sol_sim < [len_sim-1]*(N-1) \
      and not( detener )              :
      # Este bucle barre las opciones de operaciones.
      sol_sim      = siguiente( len_sim , sol_sim )
      solucion_lst = decodificar_solucion( sol_num , sol_sim , numeros , simbolos )
      solucion_str = lista_a_cadena( solucion_lst )
      sol_valida   = validaciones( sol_num , sol_sim , solucion_str )
      es_resultado = ( f_objetivo( objetivo , solucion_str ) == 0 )

    tiempo = time.time()

    if  interrupcion              \
    and tiempo - tiempos[1] >= 30 :
      # Se incluye para poder detener el proceso de fuerza bruta.
      # Se pide confirmación de continuación cada 30 segundos.
      tiempos[1] = tiempo
      A          = input("¿Desea terminar? ('S' = Salir): ")
      detener    = ( A == 'S' )

  print(f'Tiempo total transcurrido: {time.time()-tiempo_inicio}')

  if f_objetivo( objetivo , solucion_str ) == 0 :
    print(f'Se ha encontrado una solución: {solucion_str} = {int( eval( solucion_str ) )}')

  else:
    print('Proceso interrumpido sin encontrar solución.')

  return solucion_str

In [9]:
fuerza_bruta( lista_numeros , lista_operaciones , 5 , 20 , True)

Tiempo total transcurrido: 6.36752724647522
Se ha encontrado una solución: 1-2+7/3*9 = 20


'1-2+7/3*9'

## Calcula la complejidad del algoritmo por fuerza bruta.

Respuesta

------
*Este algoritmo de fuerza bruta va recorriendo todas las combinaciones de caracteres posibles.*

*Para ello, se ha contruido la función* `siguiente()` *que se encarga de obtener el siguiente elemento de forma paramétrica. Para una lista de tamaño $n$, esta función tendría potencialmente una complejidad $O(n)$ tanto para la lista de números como la de operaciones pues hay casos en los que tiene que modificar todo el vector para obtener el siguiente elemento. Técnicamente, para la lista de operaciones en realidad sería $O(n-1)$, pero eso, a todos los efectos sigue siendo $O(n)$.*

*Después se tiene la función* `decodificar_solucion()`*, esta función recorre ambas listas con un único vector, por lo que de nuevo, su complejidad sería $O(n)$.*

*Ya en el algoritmo principal, partiendo del bucle más interno, se tendría que llama a las dos funciones anteriormente mencionadas y además este bucle se repite $k^n$ veces (llamando $k$ al número de opciones de operaciones disponibles). De modo que su complejidad sería $O(k^n*(O(n)+O(n))) = O(n*k^n)$.*

*Finalmente se tendría el bucle exterior, el cual se recorre $j^n$ veces de nuevo (llamando $j$ al número de cifras disponibles) y además de tener en su interior al bucle interior, llama una vez más a* `siguiente()`*. Por este motivo su complejidad será $O(j^n*O(n*k^n))=O(n*j^n*k^n)=O(n*(j*k)^n)$.*

*Además, hay que tener en cuenta que dadas las restricciones del problema, $n$ no puede aumentar sinq ue aumenten también $j$ y $k$ al menos en la misma medida.Según este planteamiento, la complejidad de este algoritmo* `fuerza_bruta()` *queda del siguiente modo: $O(n*(j*k)^n)=O(n*(n*n)^n)=O(n^{2n+1})$.*
<br>
<br>
*Otro planteamiento pasaría por considerar las funciones* `siguiente()` *y* `decodificar_solucion()` *como instrucciones básicas. En esta caso, siguiendo el mismo razonamiento que en el caso anterior, la complejidad sería $O(n^{2n})$.*

In [ ]:
# La respuesta a esta pregunta no contiene código, únicamente texto.

## (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

------
*Este problema en realidad se trata de una búsqueda. De hecho, el problema puede explresarse en términos de un árbol de decisiones, donde la primera decisión sería por qué cifra empezar y los siguientes nodos a partir de ahí representarían combinacioens de operación y cifra.*

*Se ha decidido implementar un algoritmo de **Backtracking** combinado con **Diversificación y Poda** y **Búsqueda en Profundidad** debido a que las soluciones se encontrarán siempre en la base del árbol, cuando ya se hayan elegido las 5 cifras.*

*Para la poda se emplearán dós métodos diferentes.*

*Por un lado, la exclusión de opciones de cifras y números de forma se realizará implícita al no generar nodos con repeticiones, por lo que no será necesario esta validación.*

*Por otro lado, habrá que podar aquellos nodos cuyos valores sean no enteras. La perticularidad de este caso, es que sólo se podará el nodo si, además de tener una solución no entera, la multiplicación ya está presente en al solución parcial. Además, en caso de no estarlo, sólo se permitirá que la siguiente operación sea la multiplicación.*

*La razón por la que este algoritmo mejora el de fuerza bruta, es que no se exploran muchas de las soluciones que no proporcionan un resultado válido. Por ejemplo, en este algoritmo nunca se generan soluciones con repeticiones o cuando pese a no haber completado todas las posiciones ya se ha obtenido un resultado no entero y no se puede resolver con una multiplicación.*

*Además, para esta implementación en particular, el espacio de soluciones se recorre en el mismo orden, aunque se realice saltando expresiones que no den resultados permitidos, por lo que comparar los tiempos de ejecución permite ver la mejora para ejecuciones con los mismos valores, viéndose cláramente que el nuevo algoritmo es mucho más rápido alcanzándose la misma solución.*

In [10]:
min_max = 'MinMax'

def eliminar_repetidos( lista , solucion_lst ):
  # Esta función generaliza la eliminación de elementos repetidos.
  # Debería valer tanto para eliminar elementos únicos como listas de varios elementos.
  return list( set( lista ) - set( solucion_lst ) )

def busqueda_poda_nucleo( solucion_lst , num_lst , sim_lst , N , tiempos   \
                        , objetivo = None , min = ('',10) , max = ('',-10) \
                        , lista_completa = [], interrupcion = False ):
  division       = '/'
  multiplicacion = '*'
  error          = True
  detener        = False

  # Aquí es donde se podan a priori soluciones con elementos repetidos
  # y todo lo que no sea que la siguiente operación sea una multiplicación
  # cuando el resultado parcial no es entero.
  if  not( solucion_entera( lista_a_cadena( solucion_lst ) ) ) \
  and solucion_lst[-2] == division           :
    sim_lst_aux  = multiplicacion
  else:
    sim_lst_aux  = eliminar_repetidos( sim_lst , solucion_lst )

  num_lst_aux    = eliminar_repetidos( num_lst , solucion_lst )

  for operacion in sim_lst_aux:
    for numero in num_lst_aux:
      solucion_sal = solucion_lst + [ operacion ] + [ numero ]
      solucion_sal_str = lista_a_cadena( solucion_sal )

      if len( solucion_sal ) >= 2*N-1:
        if isinstance( objetivo , int ):
          # Sólo se comprueba el error si se tiene definido un objetivo.
          error = f_objetivo( objetivo , solucion_sal_str )

          if error == 0 :
            detener = True

        elif solucion_entera( solucion_sal_str ):
          # En caso de no tener definido un objetivo,
          # se interpreta que se buscan todas las posibilidades.
          valor = int( eval( solucion_sal_str ) )

          if objetivo == min_max:
            if min[1] > valor: min = ( solucion_sal_str , valor )
            if max[1] < valor: max = ( solucion_sal_str , valor )
          else:
            lista_completa.append( ( solucion_sal_str , valor ) )

        tiempo = time.time()

        if interrupcion:

          if (tiempo - tiempos[1]) >= 1 :
            # Se incluye para poder detener el proceso de fuerza bruta.
            # Se pide confirmación de continuación cada 30 segundos.
            tiempos[1] = tiempo
            A       = input("¿Desea terminar? ('S' = Salir): ")
            detener = ( A.upper == 'S' )


      elif solucion_entera( solucion_sal_str )        \
        or (   operacion == division              \
           and multiplicacion not in solucion_sal \
           )                                      :
        # Esta comprobación sirve para controlar cuándo explorar más en profundidad.
        # Se interrumpe bien porque ya se haya alcanzado la profundidad deseada (cantidad de números en la expresión),
        # o bien porque la solución NO es entera y no puede remediarse,
        # podando así las posibles soluciones que de aquí se derivarían.
        solucion_sal , detener , tiempos[1] , min , max , lista_completa = busqueda_poda_nucleo( solucion_sal , num_lst , sim_lst , N , tiempos , objetivo , min, max , lista_completa , interrupcion )

      if detener:
        break

    if detener:
      break

  return solucion_sal, detener , tiempos[1] , min, max , lista_completa



def busqueda_poda( num_lst , sim_lst , N , objetivo = None , interrupcion = False ):
  # En realidad, esta se encarga estríctamente de inicializaciones y de elegir el primer dígito.
  # Donde ocurre la 'mágia' es en al función a la que llama para encontrar las soluciones.
  tiempo_inicio  = time.time()
  tiempos        = [ tiempo_inicio ]*2
  detener        = False
  min            = ( '' , 0 )
  max            = ( '' , 0 )
  lista_completa = []

  for numero in num_lst:
    solucion_lst = [ numero ]
    solucion_lst , detener , tiempos[1] , min , max , lista_completa = busqueda_poda_nucleo( solucion_lst , num_lst , sim_lst , N , tiempos , objetivo , min, max , lista_completa , interrupcion )

    if detener:
      break

  solucion_str = lista_a_cadena( solucion_lst )

  print(f'Tiempo total transcurrido: {time.time()-tiempo_inicio}')

  if isinstance( objetivo , int ):
    if f_objetivo( objetivo , solucion_str ) == 0 :
      print(f'Se ha encontrado una solución: {solucion_str} = {int( eval( solucion_str ) )}')

    else:
      print('Proceso interrumpido sin encontrar solución.')

    return solucion_str

  elif objetivo == min_max:
    print(f'Mínimo : {min[0]} = {min[1]}')
    print(f'Máximo : {max[0]} = {max[1]}')

  else:
    return lista_completa

In [11]:
busqueda_poda( lista_numeros , lista_operaciones , 5 , 20 )

Tiempo total transcurrido: 0.029952287673950195
Se ha encontrado una solución: 1+3*7-4/2 = 20


'1+3*7-4/2'

## (*)Calcula la complejidad del algoritmo

Respuesta

------
*Analizando el problema por partes, se tiene que por un lado la función* `busqueda_poda_nucleo()`*, que se encarga de recorrer las diferentes opciones combinadas de cifra y operación. Tomando $k$ como el número de operaciones disponibles y $j$ el número de cifras entre las que elegir, una única ejecución de esta función (sin tener en cuenta la recursividad) tendría como complejidad $O(j*k)$.*

*Faltaría incluir la recursividad, sin embargo, cabe destacar que con cada llamada conseguitiva los j y k con los que se itera con más y más pequeños según se han ido empleando apra construir una expresión, por lo que en realidad, la complejidad de esta función sería $O(j!*k!)$.*

*Además, en realidad, la recursividad acaba teniendo como mucho una profundidad $n$, que es el tamaño del problema, por lo que no llegarían a utilizarse todos los términos de los factoriales, sino únicamente las $n$ términos más grandes de cada uno. De este modo, la complejidad anterior puede expresarse asemejarse a $O(\frac{j!}{(j-n)!}*\frac{k!}{(k-n)!})$.*

*Como ya se comentó en el apartado de la complejidad para el algoritmo de fuerza bruta, hacer crecer el tamaño del problema (es decir, la cantidad de números relacionados por expresiones) requiere también de hacer crecer la cantidad de operaciones que intervienen y de opciones de números a elegir para que sigan sin repetirse. Entonces, en el caso de que se pudieran añadir y añadir operaciones, $j$ y $k$ se acabarían comportanto como $n$, de donde se tiene que $O(\frac{j!}{(j-n)!}*\frac{k!}{(k-n)!})=O(n!^2)$.*
<br>
<br>
*Comparando las complejidades de los algoritmos* `fuerza_bruta()` *($O(n^{2n+1})$) y* `busqueda_poda_nucleo()` *($O(n!^2)$), se puede observar que el segundo es claramente menor. Tan sólo hace falta reparar en que en el primer caso, se tiene $2n+1$ términos de $n$ multiplicados entre sí, mientras que en el segundo caso, son $2n$ términos que partiendo de $n$ son cada vez más pequeños.*

In [ ]:
# La respuesta a esta pregunta no contiene código, únicamente texto.

## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

------
Se generará una lista de 10 números aleatorios entre -50 y 50, por ejemplo.

In [12]:
datos_aleatorios = [random.randint(-50,50) for i in range(10)]

print(datos_aleatorios)

[-37, -9, -29, 38, -19, 12, 29, -24, 33, 13]


## Aplica el algoritmo al juego de datos generado

Respuesta

------
Bastará con construir un bucle para ir ejecutando el agoritmo para cada uno de los valores.

In [13]:
soluciones = []

for caso in datos_aleatorios:
  soluciones.append( busqueda_poda( lista_numeros , lista_operaciones , 5 , caso ) )
  print()

print(soluciones)

Tiempo total transcurrido: 0.03172492980957031
Se ha encontrado una solución: 1+4/2-5*8 = -37

Tiempo total transcurrido: 0.0009846687316894531
Se ha encontrado una solución: 1+2-4/3*9 = -9

Tiempo total transcurrido: 0.04513144493103027
Se ha encontrado una solución: 1+6-8/2*9 = -29

Tiempo total transcurrido: 0.038275718688964844
Se ha encontrado una solución: 1+5*8-6/2 = 38

Tiempo total transcurrido: 0.03426551818847656
Se ha encontrado una solución: 1+4-6/2*8 = -19

Tiempo total transcurrido: 0.008313655853271484
Se ha encontrado una solución: 1+2*7-9/3 = 12

Tiempo total transcurrido: 0.03777742385864258
Se ha encontrado una solución: 1+5*6-4/2 = 29

Tiempo total transcurrido: 0.014522314071655273
Se ha encontrado una solución: 1+3-7/2*8 = -24

Tiempo total transcurrido: 0.033721208572387695
Se ha encontrado una solución: 1+4*9-8/2 = 33

Tiempo total transcurrido: 0.010002613067626953
Se ha encontrado una solución: 1+2*7-6/3 = 13

['1+4/2-5*8', '1+2-4/3*9', '1+6-8/2*9', '1+5*8-6/

## ¿Qué valor máximo y mínimo se pueden obtener según las condiciones del problema?

Respuesta

------
*El algoritmo se ha preparado para se capaz de devolver los valores máximos y mínimos. Este enfoque es viable para este tamaño del problema, pero no lo sería para tamaños superiores debido a que para enocntrarlo realmente se están recorriendo todas las solucones posibles.*

*A continuación se ejecutará el algoritmo de bñ´´usqueda por poda apra recorrer todas las posibilidades y recoger los extremos.*

In [14]:
busqueda_poda( lista_numeros , lista_operaciones , 5 , 'MinMax' )

Tiempo total transcurrido: 4.520033836364746
Mínimo : 1+4/2-8*9 = -69
Máximo : 7+8/1*9-2 = 77


*Para generalizar este cálculo, sería conveniente diseñar un algoritmo específico. Por ejemplo, modelando correctamente el problema se podrían diseñar algoritmos voraces que devolvieran estos resultados. Segiramente sería conveniente combinarlos con alguna estrategia, por ejemplo diviendo las soluciones en dos partes, una a maximizar y otra a minimizar debido a la presencia de la resta.*

*Estos algoritmos de tipo voraz deberían evaluar cómo varía el valor al

## ¿Es posible encontrar todos los valores enteros posibles entre dicho mínimo y máximo ?

Respuesta

------
*Para este tamaño del problema, sí sería posible. De hecho, el algoritmo está recorriendo todas las posibilidades apra encontrar los extremos. Esto no quiere decir que exista solución para todos los enteros en el intervalo, podría darse el caso de que un valor en el intervalo no fuera obtenible con estas combinaciones de operaciones.*

*El algoritmo se ha preparado apra ser capaz también de devolver esta lista.*

*Sin embargo, como se apuntaba antes, esto es factible para este tamaño de problema y para mayores tamaños el tiempo de ejecución resultaría inviable.*

In [15]:
lista_completa = busqueda_poda( lista_numeros , lista_operaciones , 5 )

Tiempo total transcurrido: 4.6967809200286865


In [16]:
print(lista_completa)

[('1+2-3/4*8', -3), ('1+2-3/6*4', 1), ('1+2-3/6*8', -1), ('1+2-3/9*6', 1), ('1+2-3*4/6', 1), ('1+2-3*6/9', 1), ('1+2-3*8/4', -3), ('1+2-3*8/6', -1), ('1+2-4/3*6', -5), ('1+2-4/3*9', -9), ('1+2-4/6*3', 1), ('1+2-4/6*9', -3), ('1+2-4/8*6', 0), ('1+2-4*3/6', 1), ('1+2-4*6/3', -5), ('1+2-4*6/8', 0), ('1+2-4*9/3', -9), ('1+2-4*9/6', -3), ('1+2-5/3*6', -7), ('1+2-5/3*9', -12), ('1+2-5/4*8', -7), ('1+2-5*6/3', -7), ('1+2-5*8/4', -7), ('1+2-5*9/3', -12), ('1+2-6/3*4', -5), ('1+2-6/3*5', -7), ('1+2-6/3*7', -11), ('1+2-6/3*8', -13), ('1+2-6/3*9', -15), ('1+2-6/4*8', -9), ('1+2-6/8*4', 0), ('1+2-6/9*3', 1), ('1+2-6*3/9', 1), ('1+2-6*4/3', -5), ('1+2-6*4/8', 0), ('1+2-6*5/3', -7), ('1+2-6*7/3', -11), ('1+2-6*8/3', -13), ('1+2-6*8/4', -9), ('1+2-6*9/3', -15), ('1+2-7/3*6', -11), ('1+2-7/3*9', -18), ('1+2-7/4*8', -11), ('1+2-7*6/3', -11), ('1+2-7*8/4', -11), ('1+2-7*9/3', -18), ('1+2-8/3*6', -13), ('1+2-8/3*9', -21), ('1+2-8/4*3', -3), ('1+2-8/4*5', -7), ('1+2-8/4*6', -9), ('1+2-8/4*7', -11), ('1+2-

## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

------
*Este trabajo no ha sido basado en ningún otro a parte de los amteriales de la asignatura.*

## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño.

Respuesta

------
*Para este problema no hay muchas opciones de variaciones. Aumentar el tamaño implicaría bien incluir nuevas operaciones o bien permitir repeticiones en al menos la lista de operadiones elegidas.*

*En el primero de los casos no supodría demasiado problema dado que por la forma en la que se ha diseñado el algoritmo y sus funciones auxiliares ya contempla estos casos.*

*En cambio, en el segundo caso, sería necesario modificar el algoritmo para que se puedan permitir las repeticiones de operaciones o números.*

*Otra de las variaciones que se podrían aportar al problema sería la de permitir números y operaciones de más de dos posiciones (como pueda ser **). Debido a la estrategia elegida de almacenar la solución en listas en vez de en un string, el algoritmo debería funcionar correctamente para este caso sin requierir nuevas modificaciones.*